In [3]:
import requests

url = 'http://localhost:5000/api/get_stories'
data = {
    'cats': 'ay.sports.nba, ay.sports.nfl',
    'word_count': '500',
    'datetime': 'NOW-1DAYS/DAY',
    'location': 'Q65',
    'writing_style': 'funny',
    'total': '5',
    'token': '344a7e67c939a28e3a250d57e3b63cf7'
}

response = requests.post(url, json=data)
response

<Response [200]>

In [2]:
data = {'cats': 'ay.sports.nba, ay.sports.nfl', 'word_count': '500', 'datetime': 'NOW-1DAYS/DAY', 'location': 'Q65', 'writing_style': 'funny', 'total': '5', 'token': '344a7e67c939a28e3a250d57e3b63cf7'}
data

{'cats': 'ay.sports.nba, ay.sports.nfl',
 'word_count': '500',
 'datetime': 'NOW-1DAYS/DAY',
 'location': 'Q65',
 'writing_style': 'funny',
 'total': '5',
 'token': '344a7e67c939a28e3a250d57e3b63cf7'}

In [3]:
# url = 'https://api.aylien.com/news/stories?
# aql=categories:({{taxonomy:aylien AND id:ay.biz.crime}} OR {{taxonomy:aylien AND id:ay.lifesoc.crime}}) 
# AND entities:({{id:Q65 AND overall_prominence:>=0.65}} OR {{id:Q30 AND overall_prominence:>=0.65}}) 
# AND sentiment.title.polarity:(negative neutral positive)
# &cursor=*&published_at.end=NOW&published_at.start=NOW-1DAYS/DAY'
categories = data['cats'].split(', ')
cat_search_string = " OR ".join([f"{{{{taxonomy: aylien AND id:{one_cat}}}}}" for one_cat in categories])
cat_search_string

'{{taxonomy: aylien AND id:ay.sports.nba}} OR {{taxonomy: aylien AND id:ay.sports.nfl}}'

In [4]:
opts = {
    'published_at_start': data['datetime'],
    'published_at_end': 'NOW',
    'language': ['en'],
    'per_page': int(data['total']),
    'source_locations_country': ['US'],
    'categories_taxonomy': 'aylien',
    'aql': f'categories: ({cat_search_string}) AND entities:({{{{id:{data["location"]} AND overall_prominence:>=0.65}}}})',
    'not_entities_id' : ['N25240567620063168378490754444048642724'], #exclude sponsored
    'sort_by': 'relevance'
}
opts

{'published_at_start': 'NOW-1DAYS/DAY',
 'published_at_end': 'NOW',
 'language': ['en'],
 'per_page': 5,
 'source_locations_country': ['US'],
 'categories_taxonomy': 'aylien',
 'aql': 'categories: ({{taxonomy: aylien AND id:ay.sports.nba}} OR {{taxonomy: aylien AND id:ay.sports.nfl}}) AND entities:({{id:Q65 AND overall_prominence:>=0.65}})',
 'not_entities_id': ['N25240567620063168378490754444048642724'],
 'sort_by': 'relevance'}

In [36]:
default_opts = {
    'published_at_start': 'NOW-2DAYS',
    'published_at_end': 'NOW',
    'language': ['en'],
    'per_page': 5,
    'source_locations_country': ['US'],
    'categories_taxonomy': 'aylien',
    'aql': 'categories: {{taxonomy: aylien AND id:ay.sports.nba}}',
    'not_entities_id' : ['N25240567620063168378490754444048642724'], #exclude sponsored
    'sort_by': 'relevance'
}

In [5]:
import os
from dotenv import load_dotenv
from __future__ import print_function
import time
import aylien_news_api
from aylien_news_api.rest import ApiException
from pprint import pprint

load_dotenv()
API_ID = os.getenv("AYLIEN_API_ID")
API_KEY = os.getenv("AYLIEN_API_KEY")

configuration = aylien_news_api.Configuration()
configuration.api_key['X-AYLIEN-NewsAPI-Application-ID'] = API_ID
configuration.api_key['X-AYLIEN-NewsAPI-Application-Key'] = API_KEY

client = aylien_news_api.ApiClient(configuration)
api_instance = aylien_news_api.DefaultApi(client)

In [6]:
try:
    api_response = api_instance.list_stories(**opts)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling DefaultApi->list_stories: %s\n" % e)

{'next_page_cursor': 'MTYuMTMxNzEyLDUzNzQ0OTQ2NTA=',
 'published_at_end': datetime.datetime(2023, 3, 7, 20, 46, 29, 946722, tzinfo=tzutc()),
 'published_at_start': datetime.datetime(2023, 3, 6, 0, 0, tzinfo=tzutc()),
 'stories': [{'author': {'avatar_url': None, 'id': 3040616, 'name': ''},
              'body': 'AVG3-Pnt.PlayerGMINFGM-FGAPCTFGM-FGAFTM-FTAPCTPTSAVGJames4736.1532-1061.501100-325224-295.759138829.5Davis4033.4394-703.56016-62253-312.811105726.4Westbrook5228.7304-729.41763-213156-238.65582715.9Russell424.819-43.4426-1710-13.7695413.5Beasley927.043-107.40226-735-8.62511713.0Walker4825.4221-49\n'
                      '2.44985-22987-100.87061412.8Schroder5130.9209-503.41660-179161-185.87063912.5Bryant4121.4197-301.65422-5080-108.74149612.1Reaves4727.5163-322.50661-156126-144.87551310.9Hachimura1724.360-138.43510-3730-43.6981609.4Vanderbilt924.229-53.5472-912-14.857728.0T.Brown5924.5160-365.43879-21227-28.9644267.2Nunn3913.5102-251.40641-12617-21.8102626.7Beverley4526.997-241.4

In [47]:
import json
# import pandas as pd

# type(api_response)     
json_response = [
    {
        'id': story.id,
        'body': story.body, 
        'author': story.author.to_dict(),
        'links': story.links.to_dict(),
        'media': [image.url for image in story.media],
        'published_at': str(story.published_at),
        'sentiment': story.sentiment.to_dict(),
        'summary': story.summary.to_dict(),
        'title': story.title,
        'words_count': story.words_count
    } 
    for story in api_response.stories]

# class CustomEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, StoryLinks):
#             return obj.to_dict()
#         return super().default(obj)
# json_response = {k: v for k, v in api_response.stories.items() if k in ('author', 'body')}
#     print(story.entities)
#         story.published_at = str(story.published_at)
#     print(story)
#         del story.entities
# response_dict = api_response.to_dict()
response_str = json.dumps(json_response)
# response_str
# response_dict['stories']
# api_response.stories
response_str
# json_response

'[{"id": 5375118923, "body": "AVG3-Pnt.PlayerGMINFGM-FGAPCTFGM-FGAFTM-FTAPCTPTSAVGJames4736.1532-1061.501100-325224-295.759138829.5Davis4033.4394-703.56016-62253-312.811105726.4Westbrook5228.7304-729.41763-213156-238.65582715.9Russell424.819-43.4426-1710-13.7695413.5Beasley927.043-107.40226-735-8.62511713.0Walker4825.4221-49\\n2.44985-22987-100.87061412.8Schroder5130.9209-503.41660-179161-185.87063912.5Bryant4121.4197-301.65422-5080-108.74149612.1Reaves4727.5163-322.50661-156126-144.87551310.9Hachimura1724.360-138.43510-3730-43.6981609.4Vanderbilt924.229-53.5472-912-14.857728.0T.Brown5924.5160-365.43879-21227-28.9644267.2Nunn3913.5102-251.40641-12617-21.8102626.7Beverley4526.997-241.40255-15839-50.7802886.4Gabriel5114.9127-208.6119-3039-63.6193025.9Bamba711.610-24.4175-156-11.545314.4Ryan1210.815-49.30613-354-5.800473.9Christie3713.744-105.41926-6214-16.8751283.5Toscano-Anderson3012.233-66.5004-2011-15.733812.7Jones228.020-37.5410-315-20.750552.5Pippen65.34-12.3331-35-9.556142.3Reed44.

In [ ]:
# json = json_response.to_json()
# json